### Import Library

In [25]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS,FamaMacBeth

In [2]:
import statsmodels.api as sm

### Import Data

In [3]:
data_directory = "data/"
file = "yahoo_data_filter_0_1.csv"

In [4]:
df = pd.read_csv(data_directory + file)
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

### Calculate the momentun signal

In [5]:
universe_df = df

In [6]:
momentum_period = 5
waiting_time = 1

In [7]:
complete_ret_sub_df = universe_df.pct_change(periods=momentum_period).shift(waiting_time).dropna()

In [8]:
complete_ret_sub_df_long = complete_ret_sub_df.stack()

### Create the future return

In [9]:
future_return_days = 1

In [10]:
future_return_df = universe_df.pct_change(future_return_days).shift(-(future_return_days-1))

In [11]:
future_return_df_long = future_return_df.stack()

### Merge Return with Signal

In [12]:
future_return_df_long = pd.DataFrame(future_return_df_long)
complete_ret_sub_df_long = pd.DataFrame(complete_ret_sub_df_long)

In [13]:
future_return_df_long.columns = ["target"]
complete_ret_sub_df_long.columns = ["signal"]

In [14]:
merge_df = future_return_df_long.merge(complete_ret_sub_df_long,left_index=True,right_index=True)

In [15]:
index_df = merge_df.index.to_frame()
index_df.columns = ["date","asset"]
index_df["target"] = merge_df["target"]
index_df["signal"] = merge_df["signal"]
index_df.index = range(0,index_df.shape[0])

In [16]:
merge_df = index_df

In [17]:
merge_df = merge_df.set_index(["asset","date"])

### Panel Regression

In [18]:
exog_vars = ["signal"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [19]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)

In [20]:
mod.fit(cov_type="clustered",cluster_entity=True,cluster_time=True)

C:\app_user_download\anaconda\lib\site-packages\linearmodels\panel\results.py:84: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


Dep. Variable:,target,R-squared:,0.0007
Estimator:,PanelOLS,R-squared (Between):,-0.0523
No. Observations:,103645,R-squared (Within):,0.0004
Date:,"Sat, Jan 09 2021",R-squared (Overall):,0.0004
Time:,16:10:09,Log-likelihood,2.83e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,73.082
Entities:,95,P-value,0.0000
Avg Obs:,1091.0,Distribution:,"F(1,102459)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,18.335


In [ ]:
mod = PanelOLS(depe,exog,entity_effects=True,time_effects=True)

In [24]:
mod.fit(cov_type="kernel",kernel="parzen",cluster_entity=True,cluster_time=True)

Dep. Variable:,target,R-squared:,0.0007
Estimator:,PanelOLS,R-squared (Between):,-0.0523
No. Observations:,103645,R-squared (Within):,0.0004
Date:,"Sat, Jan 09 2021",R-squared (Overall):,0.0004
Time:,16:14:10,Log-likelihood,2.83e+04
Cov. Estimator:,Driscoll-Kraay,,
,,F-statistic:,73.082
Entities:,95,P-value,0.0000
Avg Obs:,1091.0,Distribution:,"F(1,102459)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,19.586


### Fama Macbeth Regression

In [27]:
exog_vars = ["signal"]
depe_var = ["target"]
exog = sm.add_constant(merge_df[exog_vars])
depe = merge_df[depe_var]

In [28]:
mod = FamaMacBeth(depe,exog)

In [34]:
res = mod.fit(cov_type='kernel',kernel="bartlett")

In [35]:
res

Dep. Variable:,target,R-squared:,-7.399e-05
Estimator:,FamaMacBeth,R-squared (Between):,-0.1069
No. Observations:,103645,R-squared (Within):,2.385e-05
Date:,"Sat, Jan 09 2021",R-squared (Overall):,-7.399e-05
Time:,16:26:11,Log-likelihood,2.486e+04
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,-7.6677
Entities:,95,P-value,1.0000
Avg Obs:,1091.0,Distribution:,"F(1,103643)"
Min Obs:,1091.0,,
Max Obs:,1091.0,F-statistic (robust):,10.434
